In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import random
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define paths to your dataset folders
train_data_dir = '/content/drive/My Drive/Train'

# Image dimensions expected by ResNet50
img_height, img_width = 100, 100
num_classes = 3  # Number of classes (assuming 3 classes: SUV, Truck, Bus)

# Use ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation split
)

# Generate batches of augmented data from the directories
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # set as training data
)

# Validation data generator
valid_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # set as validation data
)

# Load pre-trained ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze all layers in the base ResNet50 model
for layer in base_model.layers:
    layer.trainable = False

# Add a global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 256 hidden units and ReLU activation
x = Dense(256, activation='relu')(x)

# Add a final softmax layer for classification
predictions = Dense(num_classes, activation='softmax')(x)

# Combine the base model and custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=1,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // valid_generator.batch_size
)

# Evaluate the model on a random image from the dataset
def evaluate_random_image(model, data_dir, img_height, img_width):
    # Get list of all classes
    classes = sorted(os.listdir(data_dir))

    # Pick a random class
    random_class = random.choice(classes)
    class_dir = os.path.join(data_dir, random_class)

    # Get list of all images in the class directory
    all_images = os.listdir(class_dir)

    # Pick a random image from the class
    random_image = random.choice(all_images)
    img_path = os.path.join(class_dir, random_image)

    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # expand dimensions to fit batch size
    img_array = img_array / 255.  # rescale to [0,1]

    # Predict class probabilities
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)

    # Display the image and predicted class
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Actual: {random_class}, Predicted: {classes[predicted_class]}')
    plt.show()

# Evaluate on a random image
evaluate_random_image(model, train_data_dir, img_height, img_width)


ValueError: mount failed